<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/index_structs/knowledge_graph/FalkorDBGraphDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="在 Colab 中打开"/></a>


# FalkorDB图存储

本笔记本介绍了如何配置`FalkorDB`作为LlamaIndex中图存储的后端。


In [ ]:
%pip install llama-index-llms-openai
%pip install llama-index-graph-stores-falkordb

In [ ]:
# 我的OpenAI密钥
import os

os.environ["OPENAI_API_KEY"] = "在这里输入API密钥"

In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)

## 使用FalkorDBGraphStore的知识图谱


### 启动FalkorDB

将FalkorDB作为图形数据库启动的最简单方法是使用[falkordb](https://hub.docker.com/r/falkordb/falkordb:edge) docker镜像。

要按照本教程的每个步骤进行操作，请按照以下方式启动镜像：

```bash
docker run -p 6379:6379 -it --rm falkordb/falkordb:edge
```


In [ ]:
from llama_index.graph_stores.falkordb import FalkorDBGraphStore

graph_store = FalkorDBGraphStore(
    "redis://localhost:6379", decode_responses=True
)

INFO:numexpr.utils:NumExpr defaulting to 8 threads.


#### 构建知识图谱


In [ ]:
from llama_index.core import SimpleDirectoryReader, KnowledgeGraphIndex

from llama_index.llms.openai import OpenAI
from llama_index.core import Settings
from IPython.display import Markdown, display

In [ ]:
documents = SimpleDirectoryReader(
    "../../../../examples/paul_graham_essay/data"
).load_data()

In [ ]:
# 定义LLM

llm = OpenAI(temperature=0, model="gpt-3.5-turbo")
Settings.llm = llm
Settings.chunk_size = 512

In [ ]:
from llama_index.core import StorageContext

storage_context = StorageContext.from_defaults(graph_store=graph_store)

# 注意：可能需要一些时间！
index = KnowledgeGraphIndex.from_documents(
    documents,
    max_triplets_per_chunk=2,
    storage_context=storage_context,
)

#### 查询知识图谱

首先，我们可以查询并将三元组发送到LLM。


In [ ]:
query_engine = index.as_query_engine(
    include_text=False, response_mode="tree_summarize"
)
response = query_engine.query(
    "Tell me more about Interleaf",
)

In [ ]:
display(Markdown(f"<b>{response}</b>"))

<b>Interleaf is a software company that was founded in 1981. It specialized in developing and selling desktop publishing software. The company's flagship product was called Interleaf, which was a powerful tool for creating and publishing complex documents. Interleaf's software was widely used in industries such as aerospace, defense, and government, where there was a need for creating technical documentation and manuals. The company was acquired by BroadVision in 2000.</b>

对于更详细的答案，我们还可以发送从中提取出检索到的三元组的文本。


In [ ]:
query_engine = index.as_query_engine(
    include_text=True, response_mode="tree_summarize"
)
response = query_engine.query(
    "Tell me more about Interleaf",
)

In [ ]:
display(Markdown(f"<b>{response}</b>"))

<b>Interleaf was a company that had smart people and built impressive technology. However, it faced challenges and eventually got crushed by Moore's Law. The exponential growth in the power of commodity processors, particularly Intel processors, in the 1990s led to the consolidation of high-end, special-purpose hardware and software companies. Interleaf was one of the casualties of this trend. While the company had talented individuals and advanced technology, it was unable to compete with the rapid advancements in processor power.</b>

#### 可视化图表


In [ ]:
%pip install pyvis

In [ ]:
## 创建图形
from pyvis.network import Network

g = index.get_networkx_graph()
net = Network(notebook=True, cdn_resources="in_line", directed=True)
net.from_nx(g)
net.show("falkordbgraph_draw.html")